# Guided Learning Approach
We have implement a series of clustering models and get some meaningful topics for now. This script helps us to allocate the renamed and re-summmarised proper topics back to each answer.  

## Get embeddings of renamed topics

In [ ]:
#read in the renamed topics
import pandas as pd
topics=pd.read_csv("06 Data analysis/04 Topic Modeling/outputs/GPT_summarization/ds1_sub_clustering_sum/m_hdb_dm100_sz60_sub_topics_info_Human_Refined.csv")

In [77]:
import io
from openai import OpenAI
from tenacity import retry, stop_after_attempt, wait_random_exponential
import numpy as np

def log_before_retry(retry_state):
    """Log detailed information before each retry."""
    exception_message = retry_state.outcome.exception() if retry_state.outcome and retry_state.outcome.exception() else 'No exception'
    print(f"Preparing for Retry {retry_state.attempt_number} due to {exception_message}")


class OpenAI_Embeddinger:
    def __init__(self,text_ds:pd.DataFrame):
        self.client =OpenAI() # Initialize client and model
        self.text_ds=text_ds.reset_index(drop=True)
        self.embedding_dict=dict()
       
    @retry(wait=wait_random_exponential(min=1, max=60),
           stop=stop_after_attempt(6),
           before_sleep=log_before_retry)
    def get_embedding_batch(self, texts,embedding_model):
        responses =  self.client.embeddings.create(input=texts, model=embedding_model)
        embeddings = [None] * len(texts)  # Initialize a list to store embeddings in order
        for response in responses.data:
                embeddings[response.index] = response.embedding  # Store the embedding using the index to maintain order
        return embeddings

    def append_embedding(self, answer_id_column_name:str,text_column_name: str, model:str,batch_size:int):
        print(batch_size)
        # Generate batches from the dataset
        self.text_ds=self.text_ds.dropna(subset=[text_column_name]).reset_index(drop=True)
        texts = self.text_ds[text_column_name].to_list()
        batches = [texts[i:i + batch_size] for i in range(0, len(texts), batch_size)]
        total_batch_num = len(batches)
        print("calling api started!")
        # Loop through each batch to process and store embeddings
        for batch_number, batch in enumerate(batches, start=1):
            start_index = (batch_number - 1) * batch_size
            end_index = start_index + len(batch)-1
            answer_names=self.text_ds.loc[start_index:end_index,answer_id_column_name]
            embeddings = self.get_embedding_batch(texts=batch, embedding_model=model)
            # we need to store the embeddings back into the dataset
            batch_dict= {name: array for name, array in zip(answer_names, embeddings)}
            self.embedding_dict.update(batch_dict)
            print(f"Batch {batch_number}/{total_batch_num} done!")

        # Indicate the end of the function with a suitable return or simply end the function
        print("All batches processed.")

In [ ]:
#call a SOTA embedding model: OpenAI's text-embedding-3-large to retrieve the vector representation of the topics 

# call_embedding_renamed_no_power=OpenAI_Embeddinger(text_ds=topics)
# call_embedding_renamed_no_power.append_embedding("topic_id","renamed_no_power","text-embedding-3-large",batch_size=20)
# embedding_no_power=call_embedding_renamed_no_power.embedding_dict


In [ ]:
# call_embedding_renamed_full_power=OpenAI_Embeddinger(text_ds=topics)
# call_embedding_renamed_full_power.append_embedding("topic_id","renamed_full_power_information","text-embedding-3-large",batch_size=20)
# embedding_full_power=call_embedding_renamed_full_power.embedding_dict

In [ ]:
# call_embedding_renamed_full_power2_refined=OpenAI_Embeddinger(text_ds=topics)
# call_embedding_renamed_full_power2_refined.append_embedding("topic_id","renamed_full_power_information2_refined","text-embedding-3-large",batch_size=20)
# embedding_full_power2_refined=call_embedding_renamed_full_power2_refined.embedding_dict

In [ ]:
# call_embedding_renamed_no_power2=OpenAI_Embeddinger(text_ds=topics)
# call_embedding_renamed_no_power2.append_embedding("topic_id","renamed_no_power2_refined","text-embedding-3-large",batch_size=20)
# embedding_no_power2_refined=call_embedding_renamed_no_power2.embedding_dict

In [ ]:
# call_embedding_renamed_no_power3=OpenAI_Embeddinger(text_ds=topics)
# call_embedding_renamed_no_power3.append_embedding("topic_id","renamed_no_power3_splitted","text-embedding-3-large",batch_size=20)
# embedding_no_power3_splitted=call_embedding_renamed_no_power3.embedding_dict

In [ ]:
# #save them 
import pickle
def pickle_write_file(object, path):
    with open(path, 'wb') as file:
        pickle.dump(object, file, protocol=pickle.HIGHEST_PROTOCOL)

# pickle_write_file(embedding_GPT,"06 Data analysis/04 Topic Modeling/outputs/embeddings/ds1_GPT_Human_summary_embedding/topic_embedding_GPT_sum_dict" )
# pickle_write_file(embedding_human,"06 Data analysis/04 Topic Modeling/outputs/embeddings/ds1_GPT_Human_summary_embedding/topic_embedding_human_renamed_dict")
# pickle_write_file(embedding_no_power, "06 Data analysis/04 Topic Modeling/outputs/embeddings/ds1_GPT_Human_summary_embedding/topic_embedding_no_power_dict")
# pickle_write_file(embedding_no_power2_refined, "06 Data analysis/04 Topic Modeling/outputs/embeddings/ds1_GPT_Human_summary_embedding/topic_embedding_no_power2_refined_dict")
# pickle_write_file(embedding_no_power3_splitted, "06 Data analysis/04 Topic Modeling/outputs/embeddings/ds1_GPT_Human_summary_embedding/topic_embedding_no_power3_splitted_dict")
# pickle_write_file(embedding_full_power, "06 Data analysis/04 Topic Modeling/outputs/embeddings/ds1_GPT_Human_summary_embedding/topic_embedding_full_power_dict")
# pickle_write_file(embedding_full_power2_refined, "06 Data analysis/04 Topic Modeling/outputs/embeddings/ds1_GPT_Human_summary_embedding/topic_embedding_full_power2_refined_dict")


In [ ]:
#the embedding of topic column Human_renamed1 and topic column GPT_summ_raw have been previously retrieved,
# we do not call them again, we just let load them 

#define functions of writing and loading with pickle 
import pickle
def pickle_load_file(path):
    with open(path, 'rb') as file:
        object=pickle.load(file)
    return object 


#load the previously reterieved embeddings 
embedding_GPT=pickle_load_file("06 Data analysis/04 Topic Modeling/outputs/embeddings/ds1_GPT_Human_summary_embedding/topic_embedding_GPT_sum_dict" )
embedding_human=pickle_load_file("06 Data analysis/04 Topic Modeling/outputs/embeddings/ds1_GPT_Human_summary_embedding/topic_embedding_human_renamed_dict")

embedding_no_power=pickle_load_file("06 Data analysis/04 Topic Modeling/outputs/embeddings/ds1_GPT_Human_summary_embedding/topic_embedding_no_power_dict")
embedding_no_power2_refined=pickle_load_file("06 Data analysis/04 Topic Modeling/outputs/embeddings/ds1_GPT_Human_summary_embedding/topic_embedding_no_power2_refined_dict")
embedding_no_power3_splitted=pickle_load_file("06 Data analysis/04 Topic Modeling/outputs/embeddings/ds1_GPT_Human_summary_embedding/topic_embedding_no_power3_splitted_dict")



embedding_full_power=pickle_load_file("06 Data analysis/04 Topic Modeling/outputs/embeddings/ds1_GPT_Human_summary_embedding/topic_embedding_full_power_dict" )
embedding_full_power2_refined=pickle_load_file( "06 Data analysis/04 Topic Modeling/outputs/embeddings/ds1_GPT_Human_summary_embedding/topic_embedding_full_power2_refined_dict")


In [85]:
#join the embedding dictionaries back to the dataset so our potential readers and reviwers can check our outputs more conveniently
topics["embedding_GPT"]=topics["topic_id"].map(embedding_GPT)
topics["embedding_human"]=topics["topic_id"].map(embedding_human)
topics["embedding_no_power"]=topics["topic_id"].map(embedding_no_power)
topics["embedding_no_power2_refined"]=topics["topic_id"].map(embedding_no_power2_refined)
topics["embedding_no_power3_splitted"]=topics["topic_id"].map(embedding_no_power3_splitted)
topics["embedding_full_power"]=topics["topic_id"].map(embedding_full_power)
topics["embedding_full_power2_refined"]=topics["topic_id"].map(embedding_full_power2_refined)

In [86]:
#change embedding in numpy array to list so that they will be fully saved in a csv table 

topics["embedding_GPT"]=[ list(x) if not np.isnan(x).any() else float('nan')  for x in topics["embedding_GPT"]]
topics["embedding_human"]=[ list(x) if not np.isnan(x).any() else float('nan') for x in topics["embedding_human"]]

In [ ]:
#also save the table 
topics.to_csv( "06 Data analysis/04 Topic Modeling/outputs/embeddings/ds1_GPT_Human_summary_embedding/ds1_topics_embedding_with_topic_text.csv",index=False )

## Pairwise relevance between each topic and each response

In [ ]:
# read in dataset1 and the embeddings of each answer
text_df=pd.read_csv("data/dataset1_text_ID_created.csv")
#join the embedding back to the dataset
text_df["embedding"]=text_df["text_id"].map(pickle_load_file("06 Data analysis/04 Topic Modeling/outputs/embeddings/ds1_text_embedding"))

In [89]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

In [ ]:
#define a function to get pairwise embedding
import re 
def get_pairwise_embedding(targeted_topic_embedding_column:str):
    topics_df=topics[["topic_id",targeted_topic_embedding_column]].dropna()
    pw_df=pd.DataFrame(cosine_similarity( np.stack(text_df["embedding"]),np.stack(topics_df[targeted_topic_embedding_column])),
             index=text_df["text_id"],
             columns=topics_df["topic_id"] ) 
    pw_df.to_csv(f"06 Data analysis/04 Topic Modeling/outputs/ds1_ex_post_guided_learning/pairwise_relevance/pw_relevance_topics{re.sub('embedding','',targeted_topic_embedding_column)}.csv")

    return pw_df

In [91]:
pw_relevance_GPT=get_pairwise_embedding("embedding_GPT" )
pw_relevance_human1=get_pairwise_embedding("embedding_human" )
pw_relevance_no_power=get_pairwise_embedding("embedding_no_power" )
pw_relevance_no_power2=get_pairwise_embedding("embedding_no_power2_refined" )
pw_relevance_no_power3=get_pairwise_embedding("embedding_no_power3_splitted" )
pw_relevance_full_power=get_pairwise_embedding("embedding_full_power" )
pw_relevance_full_power2=get_pairwise_embedding( "embedding_full_power2_refined")

In [ ]:
pickle_write_file(pw_relevance_no_power3, "06 Data analysis/04 Topic Modeling/outputs/ds1_ex_post_guided_learning/pairwise_relevance/pw_relevance_topics_no_power3_splitted.DataFrame" )